In [4]:
import pandas as pd

def analyze_turnover_win_rate(year=2025, playoffs=False):
    # Construct the correct URL based on the playoffs flag
    suffix = 'ps' if playoffs else ''
    url = f'https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/teamgame_report/year_files/all_games_{year}{suffix}.csv'
    
    # Load data
    print(url)
    df = pd.read_csv(url)
    df.drop_duplicates(subset='GameId', inplace=True)
    print(f"Total games loaded: {len(df)}")

    # Filter only games where team had better (lower) turnover factor than opponent
    better_tov = df[df['turnover_factor'] > df['opp_turnover_factor']]

    # Win rate when having better turnover factor
    win_rate = better_tov['W'].mean() * 100

    # Total unique games = half the number of rows
    total_unique_games = df.shape[0] // 2

    # Output global summary
    print(f"\nGames with better TO factor: {len(better_tov)}")
    print(f"Win rate when team had better TO factor: {win_rate:.2f}%")
    print(f"Adjusted win count: {better_tov['W'].sum()} out of estimated {total_unique_games} unique games")

    # Per-team summary
    print("\n--- Win Rate by Team (when TO factor better than opponent) ---")
    team_summary = better_tov.groupby('TEAM_NAME').agg(
        games_with_better_tov=('W', 'count'),
        wins=('W', 'sum')
    )
    team_summary['win_rate'] = team_summary['wins'] / team_summary['games_with_better_tov'] * 100
    team_summary = team_summary.sort_values(by='win_rate', ascending=False)

    print(team_summary.to_string(formatters={'win_rate': '{:.2f}%'.format}))
analyze_turnover_win_rate(2025, playoffs=False)


https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/teamgame_report/year_files/all_games_2025.csv
Total games loaded: 1230

Games with better TO factor: 591
Win rate when team had better TO factor: 66.16%
Adjusted win count: 391 out of estimated 615 unique games

--- Win Rate by Team (when TO factor better than opponent) ---
                        games_with_better_tov  wins win_rate
TEAM_NAME                                                   
San Antonio Spurs                           7     7  100.00%
Oklahoma City Thunder                      20    19   95.00%
Memphis Grizzlies                          17    15   88.24%
Los Angeles Lakers                         22    19   86.36%
Boston Celtics                             45    38   84.44%
Toronto Raptors                             5     4   80.00%
Cleveland Cavaliers                        35    27   77.14%
New York Knicks                            17    13   76.47%
Phoenix Suns                         

In [9]:
import pandas as pd


def analyze_stat_win_rate(stat_column, year=2025, playoffs=False):
    # Build URL
    suffix = 'ps' if playoffs else ''
    url = f'https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/teamgame_report/year_files/all_games_{year}{suffix}.csv'

    # Load dataset
    print(f"Loading data from: {url}")
    df = pd.read_csv(url)
    df.drop_duplicates(subset=['GameId', 'TEAM_ID'], inplace=True)
    print(f"Games loaded: {len(df)} rows")

    # Make sure stat column exists
    if stat_column not in df.columns:
        print(f"Error: Column '{stat_column}' not found.")
        return

    # Pivot games: each row = one game, columns for each team
    game_pairs = df.sort_values('TEAM_ID').groupby('GameId')

    results = []

    for game_id, group in game_pairs:
        if group.shape[0] != 2:
            continue  # skip incomplete games

        team1, team2 = group.iloc[0], group.iloc[1]

        # Compare stat
        if pd.isna(team1[stat_column]) or pd.isna(team2[stat_column]):
            continue

        if team1[stat_column] == team2[stat_column]:
            continue  # skip ties

        # Determine which team had higher stat
        higher_team = team1 if team1[stat_column] > team2[stat_column] else team2
        lower_team = team2 if higher_team is team1 else team1

        results.append({
            'TEAM_NAME': higher_team['TEAM_NAME'],
            'TEAM_ID': higher_team['TEAM_ID'],
            'won': higher_team['W'],
            stat_column: higher_team[stat_column]
        })

    results_df = pd.DataFrame(results)

    if results_df.empty:
        print("No valid games found.")
        return

    # Optional team filter
    if team_name:
        results_df = results_df[results_df['TEAM_NAME'] == team_name]
        if results_df.empty:
            print(f"No results for team '{team_name}'.")
            return

    # Summary
    avg_stat = results_df[stat_column].mean()
    win_rate = results_df['won'].mean() * 100

    print(f"\n--- Stat: {stat_column} ---")
    print(f"Games where team had higher {stat_column}: {len(results_df)}")
    print(f"Average {stat_column} in those games: {avg_stat:.2f}")
    print(f"Win rate when team had higher {stat_column}: {win_rate:.2f}%")

    if team_name is None:
        print("\n--- Team Breakdown ---")
        summary = results_df.groupby('Team').agg(
            games_with_higher_stat=('won', 'count'),
            wins=('won', 'sum'),
            avg_stat=(stat_column, 'mean')
        )
        summary['win_rate'] = summary['wins'] / summary['games_with_higher_stat'] * 100
        summary = summary.sort_values(by='win_rate', ascending=False)
        print(summary.to_string(formatters={'win_rate': '{:.2f}%'.format, 'avg_stat': '{:.2f}'.format}))

analyze_stat_win_rate('turnover_factor', year=2025, playoffs=False)


Loading data from: https://raw.githubusercontent.com/gabriel1200/player_sheets/refs/heads/master/teamgame_report/year_files/all_games_2025.csv
Games loaded: 2460 rows

--- Stat: turnover_factor ---
Games where team had higher turnover_factor: 1127
Average turnover_factor in those games: -13.68
Win rate when team had higher turnover_factor: 59.18%
